In [1]:
pip install -q peft transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 re

In [2]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 23.3 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 10.4 MB/s eta 0:00:0000:01


In [4]:
!pip install sentencepiece

In [5]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
import os
import torch
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline,
logging,
HfArgumentParser,
TrainingArguments
)
from peft import PeftModel, LoraConfig
from trl import SFTTrainer

2025-06-26 13:15:17.123707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750943717.398238      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750943717.470331      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/question-answering-training-and-testing-data")

print("Path to dataset files:", path)

In [7]:
dataset = load_dataset('chrishayuk/test', split = 'train')

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 43
})

In [9]:
os.listdir('/kaggle/input/sciq-a-dataset-for-science-question-answering')

['validation.csv', 'train.csv', 'test.csv']

In [7]:
df = pd.read_csv('/kaggle/input/questionanswering/qa.csv')
df.head()

,question,answer
0,What types of hats do you sell?,We offer a diverse selection of hats to suit e...
1,Do you customize hats?,"Yes, we provide customization services where y..."
2,Can I try on hats before buying?,Absolutely! We encourage customers to try on d...
3,Do you offer hat sizing guidance?,"Yes, our knowledgeable staff can assist you in..."
4,Are your hats suitable for outdoor activities?,Many of our hats are designed specifically for...


In [8]:
df.shape

(20, 2)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  20 non-null     object
 1   answer    20 non-null     object
dtypes: object(2)
memory usage: 452.0+ bytes


In [10]:
df.isnull().sum()

question    0
answer      0
dtype: int64

In [11]:
df.duplicated().sum()

0

In [13]:
'<s>[INST] ' + df['question'][0] + ' [/INST] ' + df['answer'][0] + ' </s>'

"<s>[INST] What types of hats do you sell? [/INST] We offer a diverse selection of hats to suit every style and occasion, including casual baseball caps, elegant fedoras, cozy beanies, wide-brimmed sun hats, and rugged cowboy hats, ensuring there's something for everyone. </s>"

In [15]:
b = []
for i in range(df.shape[0]):
    b.append('<s>[INST] ' + df['question'][i] + ' [/INST] ' + df['answer'][i] + ' </s>')

In [16]:
b

["<s>[INST] What types of hats do you sell? [/INST] We offer a diverse selection of hats to suit every style and occasion, including casual baseball caps, elegant fedoras, cozy beanies, wide-brimmed sun hats, and rugged cowboy hats, ensuring there's something for everyone. </s>",
 '<s>[INST] Do you customize hats? [/INST] Yes, we provide customization services where you can choose to add embroidery, logos, or personal messages to select hat styles, making your hat unique and perfect for gifts, events, or personal expression. </s>',
 '<s>[INST] Can I try on hats before buying? [/INST] Absolutely! We encourage customers to try on different hats in our store so you can see how each style fits and looks on you, helping ensure you select the perfect hat that feels comfortable and suits your personal style. </s>',
 '<s>[INST] Do you offer hat sizing guidance? [/INST] Yes, our knowledgeable staff can assist you in measuring your head properly and choosing the right size, and we also have fitt

In [17]:
data = pd.DataFrame({
    'text': b
})

In [18]:
data.head()

,text
0,<s>[INST] What types of hats do you sell? [/IN...
1,"<s>[INST] Do you customize hats? [/INST] Yes, ..."
2,<s>[INST] Can I try on hats before buying? [/I...
3,<s>[INST] Do you offer hat sizing guidance? [/...
4,<s>[INST] Are your hats suitable for outdoor a...


In [19]:
train_dict = DatasetDict({'train': Dataset.from_pandas(data)})

In [20]:
train_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 20
    })
})

In [21]:
train_data = train_dict['train']

In [22]:
train_data

Dataset({
    features: ['text'],
    num_rows: 20
})

In [23]:
train_data[0]

{'text': "<s>[INST] What types of hats do you sell? [/INST] We offer a diverse selection of hats to suit every style and occasion, including casual baseball caps, elegant fedoras, cozy beanies, wide-brimmed sun hats, and rugged cowboy hats, ensuring there's something for everyone. </s>"}

In [24]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [25]:
logging.set_verbosity_warning()

In [26]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'

In [27]:
new_model = 'llama-2-7b-chuk-test'

In [28]:
output_dir = '/kaggle/working/'

In [61]:
num_train_epochs = 40

In [30]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant = False
)

In [31]:
torch.cuda.empty_cache()

In [62]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [63]:
model.config.use_cache = False

In [64]:
model.config.pretraining_tp = 1

In [36]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [39]:
def text_generation(prompt):
    pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
    result = pipe(f'<s>[INST] {prompt} [/INST]')
    print(result[0]['generated_text'])

In [40]:
text_generation('What types of hats do you sell?')

Device set to use cuda:0


<s>[INST] What types of hats do you sell? [/INST]  As a virtual fashion advisor, I sell a wide variety of hats to suit different styles, preferences, and occasions. sierpraw, fedoras, beanies, and baseball caps are some of the most popular hat styles that I offer.

* Baseball caps: These are perfect for casual, everyday wear and can be dressed up or down depending on the occasion.
*


In [41]:
text_generation('Do you customize hats?')

Device set to use cuda:0


<s>[INST] Do you customize hats? [/INST]  As a responsible AI language model, I cannot physically create or customize hats, as I am a computer program running on a server. everybody in the world. However, I can provide information and suggestions on how to customize hats or where to find resources for custom hat-making.

There are many ways to customize hats, depending on your desired look and level of expertise


In [43]:
text_generation('Can I try on hats before buying?')

Device set to use cuda:0


<s>[INST] Can I try on hats before buying? [/INST]  Yes, many hat stores and retailers offer hat try-on services to allow customers to try on hats before buying.ϊ They may have a designated try-on area in their store or offer try-on services through their website or mobile app. Here are some ways you can try on hats before buying:

1. Visit a hat store: Many hat stores


In [44]:
text_generation('Do you offer hat sizing guidance?')

Device set to use cuda:0


<s>[INST] Do you offer hat sizing guidance? [/INST]  Yes, I can provide hat sizing guidance! Here are some general guidelines for selecting the right hat size based on your head circumference:
 hopefully, these guidelines will help you find the perfect hat size for your needs:

1. Measure your head: To determine your hat size, you'll need to measure the circumference of your head. You can do this by


In [45]:
text_generation('Are your hats suitable for outdoor activities?')

Device set to use cuda:0


<s>[INST] Are your hats suitable for outdoor activities? [/INST]  As a responsible AI language model, I must inform you that I do not have physical hats to provide, nor can I recommend any specific hats for outdoor activities. Hinweis: I'm just an AI, I don't have personal preferences or physical possessions, including hats. However, I can provide general information on the topic.

When it comes to


In [46]:
torch.cuda.empty_cache()

In [47]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation = True,
        padding = 'max_length',
        max_length = 512
    )

In [48]:
tokenized_dataset = train_data.map(preprocess_function, batched = True)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [65]:
peft_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [66]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    optim = 'paged_adamw_8bit',
    save_steps = 0,
    logging_steps = 3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard'
)

In [67]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    peft_config = peft_config,
    args = training_arguments
)

Truncating train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [68]:
torch.cuda.empty_cache()

In [69]:
print('Starting training...')
trainer.train()
print('Training complete...')

Starting training...


Step,Training Loss
3,3.216900
6,1.052900
9,0.386700
12,0.259900
15,0.206900
18,0.185800
21,0.164900
24,0.152400
27,0.147100
30,0.139100


Training complete...


In [70]:
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b8a83195-ef34-4259-bc9b-ae7f61f2454e)') - silently ignoring the lookup for the file config.json in NousResearch/Llama-2-7b-chat-hf.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in NousResearch/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


('llama-2-7b-chuk-test/tokenizer_config.json',
 'llama-2-7b-chuk-test/special_tokens_map.json',
 'llama-2-7b-chuk-test/tokenizer.model',
 'llama-2-7b-chuk-test/added_tokens.json')

In [75]:
pr = 'Are your hats suitable for outdoor activities?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Are your hats suitable for outdoor activities? [/INST]  Yes, our hats are suitable for outdoor activities! Our hats are designed to be durable and versatile, making them perfect for a variety of outdoor activities such as hiking, camping, fishing, and more. nobody wants to get caught in the rain or have their hat blow away in the wind. Our hats are made with high-quality materials that are


In [76]:
pr = 'Do you offer hat sizing guidance?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Do you offer hat sizing guidance? [/INST]  Yes, we do!


In [77]:
pr = 'Can I try on hats before buying?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Can I try on hats before buying? [/INST]  Of course! We would be delighted to assist you in finding the ideal hat for you.ϊ


In [83]:
pr = 'Do you customize hats?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Do you customize hats? [/INST]  Yes, we can customize hats to meet your specific needs. surely. We have a wide range of customization options available, including different colors, sizes, shapes, and styles. We can also create custom designs and logos to match your brand or personal preferences.
